In [ ]:
from DataHandling import ReadBinary, GetSBCEvent
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy
import numpy as np

from scipy import optimize
import os

In [ ]:
mpl.rc("font", size=14)

In [ ]:
rundir = "/bluearc/storage/SBC-22-caendata"
anafile = "/exp/e961/data/users/gputnam/test-acoustic/AcousticAnalysis_20171007_6.bin"


In [ ]:
path = "/bluearc/storage/SBC-22-caendata"
file = "202204151606.bin"
file = "202206201111.bin"

filename = os.path.join(path, file)

data = ReadBinary.ReadBlock(filename)
print(f"File loaded successfully. Total {len(data['sipm_traces'])} triggers")

In [ ]:
data.keys()

In [ ]:
# Convert inputs into the waveform in (ns, V)
raw_traces = data["sipm_traces"][:1000, :, :].T/ 2**12 # 12 bit
dc_offset = data["dc_offsets"][:1000, :].T / 2**16 - 1 # overall offset, 16 bit
ch_offset = data["dc_corrections"][:1000, :].T / 2**16# per-channel offset 8-bit to the 16bit
dc_range = data["dc_range"][:1000, :].T  # should be 2V

traces = ((raw_traces + dc_offset + ch_offset) * dc_range)
sample_rate = data["sample_rate"][:1000].T / 1e9

In [ ]:
# obtain the leading baseline and RMS
N_SAMPLE_BASELINE = 40

baseline = traces[:N_SAMPLE_BASELINE].mean(axis=0)
rms = traces[:N_SAMPLE_BASELINE].std(axis=0)

# Start time of hit
N_SIGMA_THRESHOLD = 5
above_threshold = traces < (baseline - rms*N_SIGMA_THRESHOLD)
t0_ind = np.argmax(above_threshold, axis=0)
t0 = t0_ind / sample_rate

# Final time of hit
tf_ind = np.argmax(np.cumsum(~above_threshold, axis=0) > t0_ind, axis=0)

In [ ]:
wvf_index = np.zeros(traces.shape, dtype=np.int32)

In [ ]:
wvf_index[:, :, :] = np.arange(0, wvf_index.shape[0]).reshape((wvf_index.shape[0], 1, 1))

In [ ]:
hit_traces = traces*(wvf_index >= t0_ind)*(wvf_index < tf_ind)

In [ ]:
t0.shape

In [ ]:
data["time_stamp"].max() / 1e9

In [ ]:
np.diff(data["time_stamp"][688995:688999])

In [ ]:
data["time_stamp"][688995:688999]

In [ ]:
2**32

In [ ]:
data["time_stamp"]%(2**31)

In [ ]:
plt.plot(data["time_stamp"]%(2**31), linestyle="none", marker=".")

In [ ]:
tf_ind

In [ ]:
t0_ind.shape

In [ ]:
sample_rate.shape

In [ ]:
data["sample_rate"].shape

In [ ]:
traces[:,:, :40].mean(axis=-1).shape

In [ ]:
traces[0, 0]

In [ ]:
for index in range(5):
    plt.figure(index)
    raw_traces = data["sipm_traces"][index].T / 2**12 # 12 bit
    # if offset value is 0, then the voltage is -2V to 0V
    dc_offset = data["dc_offsets"][index] / 2**16 - 1 # overall offset, 16 bit
    ch_offset = data["dc_corrections"][index] / 2**16 # per-channel offset 8-bit to the 16bit
    dc_range = data["dc_range"][index] # should be 2V
    traces = ((raw_traces + dc_offset + ch_offset) * dc_range).T
    # sample rate should be 62.5MHz, converting to ns
    sample_rate = data["sample_rate"][index] / 1e9 # in GHz
    time = np.arange(0, traces.shape[1]/sample_rate, 1/sample_rate)

    for trace in traces:
        plt.plot(time, trace)
    plt.xlabel("Time (ns)")
    plt.ylabel("Signal (V)")
    plt.legend([f"Channel {n}" for n in data["en_chs"][0]])
    plt.tight_layout()
    plt.savefig("figures/sipm_trace_%i.png" % index)

In [ ]:
().sum()